# Gold standard curation: Preprocessing and single-step regression

In this stage of gold standard curation, we will do the data preprocessing, selection, and single-step regression for the 153 traits in our question set. This file shows the reference steps using the trait "Breast Cancer" as an example. The workflow consists of the following steps:

1. Preprocess all the cohorts related to this trait. Each cohort should be converted to a tabular form and saved to a csv file, with columns being genetic factors, the trait, and age, gender if available;
2. If there exists at least one cohort with age or gender information, conduct regression analysis with genetic features together with age or gender as the regressors.


# 1. Basic setup

In [2]:
import os

# Set your preferred name
USER = "Shuyi"
# Set the data and output directories
DATA_ROOT = '/content/drive/MyDrive/AI4Science_Public/data'
OUTPUT_ROOT = '/content/drive/MyDrive/AI4Science_Public/output1'
TRAIT = "Kidney stones"

OUTPUT_DIR = os.path.join(OUTPUT_ROOT, USER, '-'.join(TRAIT.split()))
JSON_PATH = os.path.join(OUTPUT_DIR, "cohort_info.json")
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR, exist_ok=True)

# Gene symbol normalization may take 1-2 minutes. You may set it to False for debugging.
NORMALIZE_GENE = True

In [1]:
# This cell is only for use on Google Colab. Skip it if you run your code in other environments

import os
from google.colab import drive

drive.mount('/content/drive', force_remount=True)
proj_dir = '/content/drive/MyDrive/AI4Science_Public'
os.chdir(proj_dir)

Mounted at /content/drive


In [3]:
import sys
sys.path.append('/content/drive/MyDrive/Colab Notebooks')
!pip install mygene
!pip install sparse_lmm

# 2. Data preprocessing and selection

## 2.1. The TCGA Xena dataset

In TCGA Xena, there is either zero or one cohort related to the trait. We search the names of subdirectories to see if any matches the trait. If a match is found, we directly obtain the file paths.

In [4]:
import os
from utils import *

dataset = 'TCGA'
dataset_dir = os.path.join(DATA_ROOT, dataset)
os.listdir(dataset_dir)[:10]

['.DS_Store',
 'TCGA_Esophageal_Cancer_(ESCA)',
 'TCGA_Pheochromocytoma_Paraganglioma_(PCPG)',
 'TCGA_Bile_Duct_Cancer_(CHOL)',
 'TCGA_Kidney_Chromophobe_(KICH)',
 'TCGA_Thyroid_Cancer_(THCA)',
 'TCGA_Sarcoma_(SARC)',
 'TCGA_Adrenocortical_Cancer_(ACC)',
 'TCGA_Thymoma_(THYM)',
 'TCGA_Glioblastoma_(GBM)']

If no match is found, jump directly to GEO in Part 2.2

In [ ]:
trait_subdir = "TCGA_Breast_Cancer_(BRCA)"
cohort = 'Xena'
# All the cancer traits in Xena are binary
trait_type = 'binary'
# Once a relevant cohort is found in Xena, we can generally assume the gene and clinical data are available
is_available = True

clinical_data_file = os.path.join(dataset_dir, trait_subdir, 'TCGA.BRCA.sampleMap_BRCA_clinicalMatrix')
genetic_data_file = os.path.join(dataset_dir, trait_subdir, 'TCGA.BRCA.sampleMap_HiSeqV2_PANCAN.gz')

In [ ]:
import pandas as pd

clinical_data = pd.read_csv(clinical_data_file, sep='\t', index_col=0)
genetic_data = pd.read_csv(genetic_data_file, compression='gzip', sep='\t', index_col=0)
age_col = gender_col = None

In [ ]:
_, clinical_data_cols = check_rows_and_columns(clinical_data)

In [ ]:
clinical_data_cols[:10]

['AJCC_Stage_nature2012',
 'Age_at_Initial_Pathologic_Diagnosis_nature2012',
 'CN_Clusters_nature2012',
 'Converted_Stage_nature2012',
 'Days_to_Date_of_Last_Contact_nature2012',
 'Days_to_date_of_Death_nature2012',
 'ER_Status_nature2012',
 'Gender_nature2012',
 'HER2_Final_Status_nature2012',
 'Integrated_Clusters_no_exp__nature2012']

Read all the column names in the clinical dataset, to find the columns that record information about age or gender.
Reference prompt:

In [ ]:
f'''
Below is a list of column names from a biomedical dataset. Please examine it and identify the columns that are likely to contain information about patients' age. Additionally, please do the same for columns that may hold data on patients' gender. Please provide your answer by strictly following this format, without redundant words:
candidate_age_cols = [col_name1, col_name2, ...]
candidate_gender_cols = [col_name1, col_name2, ...]
If no columns match a criterion, please provide an empty list.

Column names:
{clinical_data_cols}
'''

"\nBelow is a list of column names from a biomedical dataset. Please examine it and identify the columns that are likely to contain information about patients' age. Additionally, please do the same for columns that may hold data on patients' gender. Please provide your answer by strictly following this format, without redundant words:\ncandidate_age_cols = [col_name1, col_name2, ...]\ncandidate_gender_cols = [col_name1, col_name2, ...]\nIf no columns match a criterion, please provide an empty list.\n\nColumn names:\n['AJCC_Stage_nature2012', 'Age_at_Initial_Pathologic_Diagnosis_nature2012', 'CN_Clusters_nature2012', 'Converted_Stage_nature2012', 'Days_to_Date_of_Last_Contact_nature2012', 'Days_to_date_of_Death_nature2012', 'ER_Status_nature2012', 'Gender_nature2012', 'HER2_Final_Status_nature2012', 'Integrated_Clusters_no_exp__nature2012', 'Integrated_Clusters_unsup_exp__nature2012', 'Integrated_Clusters_with_PAM50__nature2012', 'Metastasis_Coded_nature2012', 'Metastasis_nature2012', '

In [ ]:
candidate_age_cols = ['Age_at_Initial_Pathologic_Diagnosis_nature2012', 'age_at_initial_pathologic_diagnosis',
                      'days_to_birth', 'year_of_initial_pathologic_diagnosis']
candidate_gender_cols = ['Gender_nature2012', 'gender']

Choose a single column from the candidate columns that record age and gender information respectively.
If no column meets the requirement, keep 'age_col' or 'gender_col' to None

In [ ]:
preview_df(clinical_data[candidate_age_cols])

{'Age_at_Initial_Pathologic_Diagnosis_nature2012': [nan, nan, nan, nan, nan],
 'age_at_initial_pathologic_diagnosis': [55.0, 50.0, 62.0, 52.0, 50.0],
 'days_to_birth': [-20211.0, -18538.0, -22848.0, -19074.0, -18371.0],
 'year_of_initial_pathologic_diagnosis': [2004.0,
  2003.0,
  2011.0,
  2011.0,
  2013.0]}

In [ ]:
age_col = 'age_at_initial_pathologic_diagnosis'

In [ ]:
preview_df(clinical_data[candidate_gender_cols])

{'Gender_nature2012': [nan, nan, nan, nan, nan],
 'gender': ['FEMALE', 'FEMALE', 'FEMALE', 'FEMALE', 'FEMALE']}

In [ ]:
gender_col = 'gender'

In [ ]:
selected_clinical_data = xena_select_clinical_features(clinical_data, TRAIT, age_col, gender_col)

In [ ]:
if NORMALIZE_GENE:
    genetic_data = normalize_gene_symbols_in_index(genetic_data)

In [ ]:
merged_data = selected_clinical_data.join(genetic_data.T).dropna()
merged_data.head()

,Breast Cancer,Age,Gender,ARHGEF10L,HIF3A,RNF17,RNF10,RNF11,RNF13,GTF2IP1,...,TULP2,NPY5R,GNGT2,GNGT1,TULP3,PTRF,BCL6B,GSTK1,SELP,SELS
sampleID,,,,,,,,,,,,,,,,,,,,,
TCGA-3C-AAAU-01,1,55.0,0.0,0.607308,-3.194126,-0.531035,-0.145872,0.237422,-0.29921,-0.142694,...,-0.748878,0.565583,-0.767233,-1.28139,-0.271377,-0.492286,0.360373,1.067905,0.076267,-0.392212
TCGA-3C-AALI-01,1,50.0,0.0,-0.641192,-4.928226,0.095465,0.098128,-0.541978,-0.32291,-0.044694,...,0.312922,0.079683,0.475267,1.99661,0.161423,0.273714,0.644673,-0.428695,0.068667,-0.043812
TCGA-3C-AALJ-01,1,62.0,0.0,1.082808,-4.623726,-0.531035,0.484028,-0.183678,-0.91901,0.261106,...,-0.748878,-0.656117,-0.216733,0.21081,0.101023,0.660514,1.295073,0.915105,0.168567,0.047788
TCGA-3C-AALK-01,1,52.0,0.0,0.121608,-2.881526,-0.531035,0.179128,0.039222,-0.45491,0.180306,...,0.415422,-0.178417,-0.211233,-1.28139,0.108023,1.132814,0.496773,0.240105,3.099767,0.112888
TCGA-4H-AAAK-01,1,50.0,0.0,0.420208,-3.282726,-0.531035,-0.020972,-0.117978,-0.55781,-0.173794,...,0.685222,-0.698717,-0.484233,-1.28139,-0.223577,1.222714,0.225573,-0.301995,0.200067,-0.080212


In [ ]:
print(f"The merged dataset contains {len(merged_data)} samples.")

The merged dataset contains 1215 samples.


In [ ]:
is_trait_biased, merge_data = judge_and_remove_biased_features(merged_data, TRAIT, trait_type=trait_type)
is_trait_biased

For the feature 'Breast Cancer', the least common label is '0' with 113 occurrences. This represents 9.30% of the dataset.
The distribution of the feature 'Breast Cancer' in this dataset is fine.

Quartiles for 'Age':
  25%: 48.0
  50% (Median): 58.0
  75%: 67.0
Min: 26.0
Max: 90.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '1.0' with 13 occurrences. This represents 1.07% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.



False

In [ ]:
merged_data.head()
if not is_trait_biased:
    merge_data.to_csv(os.path.join(OUTPUT_DIR, cohort+'.csv'), index=False)

In [ ]:
save_cohort_info(cohort, JSON_PATH, is_available, is_trait_biased, merged_data)

## 2.2. The GEO dataset

In GEO, there may be one or multiple cohorts for a trait. Each cohort is identified by an accession number. We iterate over all accession numbers in the corresponding subdirectory, preprocess the cohort data, and save them to csv files.

In [ ]:
dataset = 'GEO'
trait_subdir = "Kidney-stones"

trait_path = os.path.join(DATA_ROOT, dataset, trait_subdir)
os.listdir(trait_path)

['GSE73680', 'GSE11138', 'GSE123993']

Repeat the below steps for all the accession numbers

In [ ]:
cohort = accession_num = "GSE73680"
cohort_dir = os.path.join(trait_path, accession_num)
soft_file, matrix_file = get_relevant_filepaths(cohort_dir)
soft_file, matrix_file

('/content/drive/MyDrive/AI4Science_Public/data/GEO/Kidney-stones/GSE73680/GSE73680_family.soft.gz',
 '/content/drive/MyDrive/AI4Science_Public/data/GEO/Kidney-stones/GSE73680/GSE73680_series_matrix.txt.gz')

### Inital filtering and clinical data preprocessing

In [ ]:
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']

background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
print(background_info)

!Series_title	"Gene expression profile of Randall's Plaque tissue"
!Series_summary	"Randall’s plaque (RP) is the origin of renal calcification on which idiopathic calcium oxalate (CaOx) kidney stones develop. To establish genomic pathogenesis of RP, we performed the microarray analysis for comparing the gene expressions among renal papillary RP and normal tissue of 23 CaOx and 6 calcium phosphate (CaP) stone formers, and normal papillary tissue of 7 control patients. Compare to normal papillary tissue, RP tissue contained up-regulation of lipocalin 2, interleukin 11, prostaglandin-endoperoxide synthase 1, glutathione peroxidase 3, and monocyte to macrophage differentiation, whereas down-regulation of solute carrier family 12 member 1  and sodium leak channel non selective (either > 2.0- or 0.5-fold, p <0.01). The network and toxicity analysis showed these genes had association with activated mitogen-activated protein kinase, Akt/ phosphatidylinositol 3-kinase pathway, and pro-inflammat

In [ ]:
clinical_data.head()

,!Sample_geo_accession,GSM1900673,GSM1900674,GSM1900675,GSM1900676,GSM1900677,GSM1900678,GSM1900679,GSM1900680,GSM1900681,...,GSM1900725,GSM1900726,GSM1900727,GSM1900728,GSM1900729,GSM1900730,GSM1900731,GSM1900732,GSM1900733,GSM1900734
0,!Sample_characteristics_ch1,gender: female,gender: male,gender: female,gender: female,gender: male,gender: male,gender: male,gender: male,gender: male,...,gender: female,gender: male,gender: male,gender: female,gender: female,gender: female,gender: female,gender: female,gender: male,gender: male
1,!Sample_characteristics_ch1,lesion: Normal mucosa,lesion: Normal mucosa,lesion: Normal mucosa,lesion: Plaque mucosa,lesion: Normal mucosa,lesion: Plaque mucosa,lesion: Normal mucosa,lesion: Plaque mucosa,lesion: Plaque mucosa,...,lesion: Plaque mucosa,lesion: Normal mucosa,lesion: Plaque mucosa,lesion: Normal mucosa,lesion: Plaque mucosa,lesion: Normal mucosa,lesion: Plaque mucosa,lesion: Normal mucosa,lesion: Normal mucosa,lesion: Plaque mucosa
2,!Sample_characteristics_ch1,tissue type: normal papillary tissue from cont...,tissue type: normal papillary tissue from cont...,tissue type: normal papillary tissue from calc...,tissue type: Randall's Plaque from calcium stone,tissue type: normal papillary tissue from calc...,tissue type: Randall's Plaque from calcium stone,tissue type: normal papillary tissue from calc...,tissue type: Randall's Plaque from calcium stone,tissue type: Randall's Plaque from calcium stone,...,tissue type: Randall's Plaque from calcium stone,tissue type: normal papillary tissue from calc...,tissue type: Randall's Plaque from calcium stone,tissue type: normal papillary tissue from calc...,tissue type: Randall's Plaque from calcium stone,tissue type: normal papillary tissue from calc...,tissue type: Randall's Plaque from calcium stone,tissue type: normal papillary tissue from cont...,tissue type: normal papillary tissue from calc...,tissue type: Randall's Plaque from calcium stone


In [ ]:
clinical_data_unique = get_unique_values_by_row(clinical_data)
clinical_data_unique

{0: ['gender: female', 'gender: male'],
 1: ['lesion: Normal mucosa', 'lesion: Plaque mucosa'],
 2: ['tissue type: normal papillary tissue from control patients without any kidney stone',
  'tissue type: normal papillary tissue from calcium stone',
  "tissue type: Randall's Plaque from calcium stone"]}

Analyze the metadata to determine data relevance and find ways to extract the clinical data.
Reference prompt:

In [ ]:
f'''As a biomedical research team, we are selecting datasets to study the association between the human trait \'{TRAIT}\' and genetic factors, optionally considering the influence of age and gender. After searching the GEO database and parsing the matrix file of a series, we obtained background information and sample characteristics data. We will provide textual information about the dataset background, and a Python dictionary storing a list of unique values for each field of the sample characteristics data. Please carefully review the provided information and answer the following questions about this dataset:
1. Does this dataset contain gene expression data? (Note: Pure miRNA data is not suitable.)
2. For each of the traits \'{TRAIT}\', 'age', and 'gender', please address these points:
   (1) Is there human data available for this trait?
   (2) If so, identify the key in the sample characteristics dictionary where unique values of this trait is recorded. The key is an integer. The trait information might be explicitly recorded, or can be inferred from the field with some biomedical knowledge or understanding about the data collection process.
   (3) Choose an appropriate data type (either 'continuous' or 'binary') for each trait. Write a Python function to convert any given value of the trait to this data type. The function should handle inference about the trait value and convert unknown values to None.
   Name the functions 'convert_trait', 'convert_age', and 'convert_gender', respectively.

Background information about the dataset:
{background_info}

Sample characteristics dictionary (from "!Sample_characteristics_ch1", converted to a Python dictionary that stores the unique values for each field):
{clinical_data_unique}
'''

'As a biomedical research team, we are selecting datasets to study the association between the human trait \'Kidney stones\' and genetic factors, optionally considering the influence of age and gender. After searching the GEO database and parsing the matrix file of a series, we obtained background information and sample characteristics data. We will provide textual information about the dataset background, and a Python dictionary storing a list of unique values for each field of the sample characteristics data. Please carefully review the provided information and answer the following questions about this dataset:\n1. Does this dataset contain gene expression data? (Note: Pure miRNA data is not suitable.)\n2. For each of the traits \'Kidney stones\', \'age\', and \'gender\', please address these points:\n   (1) Is there human data available for this trait?\n   (2) If so, identify the key in the sample characteristics dictionary where unique values of this trait is recorded. The key is a

Understand and verify the answer from GPT, to assign values to the below variables. Assign None to the 'row_id' variables if relevant data row was not found.
Later we need to let GPT format its answer to automatically do these. But given the complexity of this step, let's grow some insight from the free-text answers for now.

In [ ]:
is_gene_availabe = True
trait_row_id = None
age_row_id = None
gender_row_id = 1

trait_type = 'binary'

In [ ]:
is_available = is_gene_availabe and (trait_row_id is not None)
if not is_available:
    save_cohort_info(cohort, JSON_PATH, is_available)
    print("This cohort is not usable. Please skip the following steps and jump to the next accession number.")

This cohort is not usable. Please skip the following steps and jump to the next accession number.


In [ ]:
# Verify and use the functions generated by GPT

def convert_trait(value):
  if "Randall's Plaque" in value:
    return 1 # Presence of kidney stones
  else:
    return 0 # Absence of kidney stones or unknown



def convert_age(value):
    # For 'age': Continuous
    try:
        return float(value)  # Assuming age is represented as a numerical value
    except ValueError:
        return None  # Unable to convert to a numerical value, treat as unknown


def convert_gender(value):
    # For 'gender': Binary
    if 'female' in value.lower():
        return '1'
    elif 'male' in value.lower():
        return '0'
    else:
        return None  # Unknown gender



In [ ]:
selected_clinical_data = geo_select_clinical_features(clinical_data, TRAIT, trait_row_id, age_row_id, gender_row_id,
                                                      convert_trait, convert_age, convert_gender)
selected_clinical_data.head()

,GSM1900673,GSM1900674,GSM1900675,GSM1900676,GSM1900677,GSM1900678,GSM1900679,GSM1900680,GSM1900681,GSM1900682,...,GSM1900725,GSM1900726,GSM1900727,GSM1900728,GSM1900729,GSM1900730,GSM1900731,GSM1900732,GSM1900733,GSM1900734
Kidney stones,0,0,0,1,0,1,0,1,1,0,...,1,0,1,0,1,0,1,0,0,1
Gender,1,0,1,1,0,0,0,0,0,0,...,1,0,0,1,1,1,1,1,0,0


### Genetic data preprocessing and final filtering

In [ ]:
genetic_data = get_genetic_data(matrix_file)
genetic_data.head()

,ID,GSM1900673,GSM1900674,GSM1900675,GSM1900676,GSM1900677,GSM1900678,GSM1900679,GSM1900680,GSM1900681,...,GSM1900725,GSM1900726,GSM1900727,GSM1900728,GSM1900729,GSM1900730,GSM1900731,GSM1900732,GSM1900733,GSM1900734
0,(+)E1A_r60_1,0.349052,-0.153754,0.362405,0.242259,1.485898,0.886411,1.910482,1.224578,1.015731,...,1.465079,-0.675628,-0.995711,-1.081307,0.372525,-0.549891,-1.047817,-1.222464,-1.663074,-0.953308
1,(+)E1A_r60_3,-0.762601,-0.663358,0.703987,0.745912,-0.063095,0.205520,-0.339061,0.444654,0.714867,...,-0.153791,0.933729,-0.845052,-0.040140,0.256806,0.105194,-0.646615,0.011775,-0.126527,-0.411852
2,(+)E1A_r60_a104,-0.609916,-0.907433,-0.044035,-0.016203,-0.123792,-0.178413,-0.045962,0.118390,0.326883,...,1.166544,1.344295,-0.127290,0.545580,1.020490,0.719269,-0.101138,0.651258,0.253140,0.044324
3,(+)E1A_r60_a107,-0.434986,-0.310954,0.421687,0.528773,0.781375,1.064137,1.706558,1.079957,1.036660,...,3.058856,0.761079,-0.647829,-0.505336,0.911798,0.053446,-0.854421,0.137606,-0.258092,-0.603940
4,(+)E1A_r60_a135,0.252605,-0.136379,0.184774,0.196707,1.920272,1.153875,2.381676,1.645191,1.545880,...,3.737506,-0.580734,-1.034089,-0.789300,0.804862,-0.421929,-0.888240,-1.207691,-1.102594,-1.050342


In [ ]:
gene_row_ids = genetic_data['ID'][:20].tolist()
gene_row_ids

['(+)E1A_r60_1',
 '(+)E1A_r60_3',
 '(+)E1A_r60_a104',
 '(+)E1A_r60_a107',
 '(+)E1A_r60_a135',
 '(+)E1A_r60_a20',
 '(+)E1A_r60_a22',
 '(+)E1A_r60_a97',
 '(+)E1A_r60_n11',
 '(+)E1A_r60_n9',
 '3xSLv1',
 'A_19_P00315452',
 'A_19_P00315459',
 'A_19_P00315482',
 'A_19_P00315492',
 'A_19_P00315493',
 'A_19_P00315502',
 'A_19_P00315506',
 'A_19_P00315518',
 'A_19_P00315519']

Check if the gene dataset requires mapping to get the gene symbols corresponding to each data row.

Reference prompt:

In [ ]:
f'''
Below are the row headers of a gene expression dataset in GEO. Based on your biomedical knowledge, are they human gene symbols, or are they some other identifiers that need to be mapped to gene symbols? Your answer should be concluded by starting a new line and strictly following this format:
requires_gene_mapping = (True or False)

Row headers:
{gene_row_ids}
'''

"\nBelow are the row headers of a gene expression dataset in GEO. Based on your biomedical knowledge, are they human gene symbols, or are they some other identifiers that need to be mapped to gene symbols? Your answer should be concluded by starting a new line and strictly following this format:\nrequires_gene_mapping = (True or False)\n\nRow headers:\n['(+)E1A_r60_1', '(+)E1A_r60_3', '(+)E1A_r60_a104', '(+)E1A_r60_a107', '(+)E1A_r60_a135', '(+)E1A_r60_a20', '(+)E1A_r60_a22', '(+)E1A_r60_a97', '(+)E1A_r60_n11', '(+)E1A_r60_n9', '3xSLv1', 'A_19_P00315452', 'A_19_P00315459', 'A_19_P00315482', 'A_19_P00315492', 'A_19_P00315493', 'A_19_P00315502', 'A_19_P00315506', 'A_19_P00315518', 'A_19_P00315519']\n"

If not required, jump directly to the gene normalization step

In [ ]:
requires_gene_mapping = True

In [ ]:
if requires_gene_mapping:
    gene_annotation = get_gene_annotation(soft_file)[0]
    gene_annotation_summary = preview_df(gene_annotation)
    gene_annotation_summary

Observe the first few cells in the ID column of the gene annotation dataframe, to find the names of columns that store the gene probe IDs and gene symbols respectively.
Reference prompt:

In [ ]:
if requires_gene_mapping:
    print(f'''
    As a biomedical research team, we extracted the gene annotation data from a series in the GEO database, and saved it to a Python dictionary. Please read the dictionary, and decide which key stores the ID of the probe, and which key stores the gene symbols. Please strict follow this format in your answer:
    probe_name_key = key_name1
    gene_name_key = key_name2

    Gene annotation dictionary:
    {gene_annotation_summary}
    ''')


    As a biomedical research team, we extracted the gene annotation data from a series in the GEO database, and saved it to a Python dictionary. Please read the dictionary, and decide which key stores the ID of the probe, and which key stores the gene symbols. Please strict follow this format in your answer:
    probe_name_key = key_name1
    gene_name_key = key_name2

    Gene annotation dictionary:
    {'ID': ['GE_BrightCorner', 'DarkCorner', 'A_23_P117082', 'A_33_P3246448', 'A_33_P3318220'], 'SPOT_ID': ['CONTROL', 'CONTROL', 'A_23_P117082', 'A_33_P3246448', 'A_33_P3318220'], 'CONTROL_TYPE': ['pos', 'pos', 'FALSE', 'FALSE', 'FALSE'], 'REFSEQ': [nan, nan, 'NM_015987', 'NM_080671', 'NM_178466'], 'GB_ACC': [nan, nan, 'NM_015987', 'NM_080671', 'NM_178466'], 'LOCUSLINK_ID': [nan, nan, 50865.0, 23704.0, 128861.0], 'GENE_SYMBOL': [nan, nan, 'HEBP1', 'KCNE4', 'BPIFA3'], 'GENE_NAME': [nan, nan, 'heme binding protein 1', 'potassium voltage-gated channel, Isk-related family, member 4', 'BPI fo

In [ ]:
if requires_gene_mapping:
    probe_id_key = 'ID'
    gene_symb_key = 'GENE_SYMBOL'
    gene_mapping = get_gene_mapping(gene_annotation, probe_id_key, gene_symb_key)
    genetic_data = apply_gene_mapping(genetic_data, gene_mapping)

In [ ]:
if NORMALIZE_GENE:
    genetic_data = normalize_gene_symbols_in_index(genetic_data)

INFO:biothings.client:querying 1-1000...
INFO:biothings.client:done.
INFO:biothings.client:Finished.
INFO:biothings.client:Pass "returnall=True" to return complete lists of duplicate or missing query terms.
INFO:biothings.client:querying 1-1000...
INFO:biothings.client:done.
INFO:biothings.client:Finished.
INFO:biothings.client:Pass "returnall=True" to return complete lists of duplicate or missing query terms.
INFO:biothings.client:querying 1-1000...
INFO:biothings.client:done.
INFO:biothings.client:Finished.
INFO:biothings.client:Pass "returnall=True" to return complete lists of duplicate or missing query terms.
INFO:biothings.client:querying 1-1000...
INFO:biothings.client:done.
INFO:biothings.client:Finished.
INFO:biothings.client:Pass "returnall=True" to return complete lists of duplicate or missing query terms.
INFO:biothings.client:querying 1-1000...
INFO:biothings.client:done.
INFO:biothings.client:Finished.
INFO:biothings.client:Pass "returnall=True" to return complete lists of

In [ ]:
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, genetic_data)
# The preprocessing runs through, which means is_available should be True
is_available = True

In [ ]:
print(f"The merged dataset contains {len(merged_data)} samples.")

The merged dataset contains 62 samples.


In [ ]:
is_trait_biased, merged_data = judge_and_remove_biased_features(merged_data, TRAIT, trait_type=trait_type)
is_trait_biased

For the feature 'Kidney stones', the least common label is '1' with 29 occurrences. This represents 46.77% of the dataset.
The distribution of the feature 'Kidney stones' in this dataset is fine.

For the feature 'Gender', the least common label is '1' with 22 occurrences. This represents 35.48% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.



False

In [ ]:
save_cohort_info(cohort, JSON_PATH, is_available, is_trait_biased, merged_data)

A new JSON file was created at: /content/drive/MyDrive/AI4Science_Public/output1/Shuyi/Kidney-stones/cohort_info.json


In [ ]:
merged_data.head()
if not is_trait_biased:
    merged_data.to_csv(os.path.join(OUTPUT_DIR, cohort+'.csv'), index=False)

### 3. Do regression & Cross Validation

In [5]:
best_cohort, ranked_df = filter_and_rank_cohorts(JSON_PATH, 'has_gender')
best_cohort

'GSE73680'

In [6]:
ranked_df.head()

,cohort_id,is_usable,is_available,is_biased,has_age,has_gender,sample_size,note
0,GSE73680,True,True,False,False,True,62.0,


In [7]:
merged_data = pd.read_csv(os.path.join(OUTPUT_DIR, best_cohort+'.csv'))

In [8]:
merged_data.head()

,Kidney stones,Gender,A1BG,A1BG-AS1,A1CF,A2M,A2ML1,A2MP1,A4GALT,A4GNT,...,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3
0,0,1,-0.541055,0.194566,-0.782465,1.057854,0.818379,-1.027517,0.226329,0.294019,...,-0.082772,-1.186609,-1.099665,1.005918,-0.244072,-1.642007,0.379947,1.178340,-0.334306,0.727532
1,0,0,-1.224513,0.426283,-1.308139,0.512311,-1.038083,-0.342486,0.227633,-1.981644,...,-0.156997,-0.976114,-0.793140,1.396319,0.291947,-1.500572,0.927806,0.926126,0.466821,1.055147
2,0,1,-0.527255,0.711320,-0.578092,0.077415,-0.141485,-0.025948,0.195994,0.920846,...,0.385947,-0.389178,-0.325871,1.271164,0.814773,-0.446299,0.644899,0.531508,1.185137,0.843742
3,1,1,-0.275806,0.702709,0.341932,1.521002,0.090934,-0.367957,0.325779,1.028278,...,0.135968,0.523924,-0.758016,0.277671,0.262048,-0.649630,-0.212716,0.931173,0.286901,0.098785
4,0,0,-1.781846,-0.451157,-0.704205,-0.386456,0.158411,-0.094213,0.546055,0.622370,...,0.335590,-1.076357,-0.716868,1.001984,0.557529,-0.919691,0.493604,0.047020,0.248961,0.609617


In [9]:
# If both age and gender features are available, select 'age' as the condition.
condition = 'Gender'
# Remove the other condition to prevent interference.
#merged_data = merged_data.drop(columns=['Gender'], errors='ignore').astype('float')

In [10]:
X = merged_data.drop(columns=[TRAIT, condition]).values
Y = merged_data[TRAIT].values
Z = merged_data[condition].values

Select the appropriate regression model depending on whether the dataset shows batch effect.

In [11]:
has_batch_effect = detect_batch_effect(X)
has_batch_effect

True

In [12]:
# Select appropriate models based on whether the dataset has batch effect.
# We experiment on two models for each branch. We will decide which one to choose later.

if has_batch_effect:
    model_constructor1 = VariableSelection
    model_params1 = {'modified': True, 'lamda': 3e-4}
    model_constructor2 = VariableSelection
    model_params2 = {'modified': False}
else:
    model_constructor1 = Lasso
    model_params1 = {'alpha': 1.0, 'random_state': 42}
    model_constructor2 = VariableSelection
    model_params2 = {'modified': False}

In [13]:
trait_type = 'binary'  # Remember to set this properly, either 'binary' or 'continuous'
cv_mean1, cv_std1 = cross_validation(X, Y, Z, model_constructor1, model_params1, target_type=trait_type)

alpha for Lasso: 0.0003
alpha for Lasso: 0.0003
alpha for Lasso: 0.0003
alpha for Lasso: 0.0003
alpha for Lasso: 0.0003
The cross-validation accuracy is 51.67% ± 6.24%


In [14]:
cv_mean2, cv_std2 = cross_validation(X, Y, Z, model_constructor2, model_params2, target_type=trait_type)

The cross-validation accuracy is 56.67% ± 9.72%


In [15]:
normalized_X, _ = normalize_data(X)
normalized_Z, _ = normalize_data(Z)

# Train regression model on the whole dataset to identify significant genes
model1 = ResidualizationRegressor(model_constructor1, model_params1)
model1.fit(normalized_X, Y, normalized_Z)

model2 = ResidualizationRegressor(model_constructor2, model_params2)
model2.fit(normalized_X, Y, normalized_Z)

alpha for Lasso: 0.0003


### 5. Discussion and report

In [16]:
feature_cols = merged_data.columns.tolist()
feature_cols.remove(TRAIT)

threshold = 0.05
interpret_result(model1, feature_cols, TRAIT, condition, threshold=threshold, save_output=True,
                 output_dir=OUTPUT_DIR, model_id=1)

Effect of the condition on the target variable:
Variable: Gender
Coefficient: -0.0435
Found 51 genes with non-zero coefficients associated with the trait 'Kidney stones' conditional on the factor 'Gender'. These genes are identified as significant based on the regression model.
    Variable  Coefficient  Absolute Coefficient
         JPX    -9.866888              9.866888
       NPHS1     9.393107              9.393107
        PAK3     5.572077              5.572077
       KCNK1     5.403920              5.403920
      SPATA9    -4.916649              4.916649
      CSRNP2    -4.893371              4.893371
       CSAG2     4.883998              4.883998
    SLC22A16     4.543259              4.543259
LOC100129148    -4.100822              4.100822
       PDE8B    -4.087719              4.087719
        SIX2     3.550625              3.550625
      ZNF345     3.461122              3.461122
 SNORD116-28    -3.257752              3.257752
        NGDN    -3.220438              3.220438
 

/content/drive/MyDrive/Colab Notebooks/utils.py:434: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gene_regression_df['Absolute Coefficient'] = gene_regression_df['Coefficient'].abs()


In [17]:
interpret_result(model2, feature_cols, TRAIT, condition, threshold=threshold, save_output=True,
                 output_dir=OUTPUT_DIR, model_id=2)

Effect of the condition on the target variable:
Variable: Gender
Coefficient: -0.0435
Found 0 significant genes associated with the trait 'Kidney stones' conditional on the factor 'Gender', with corrected p-value < 0.05:
Empty DataFrame
Columns: [Variable, Coefficient, p_value, corrected_p_value]
Index: []


/content/drive/MyDrive/Colab Notebooks/utils.py:443: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gene_regression_df.loc[:, 'corrected_p_value'] = corrected_p_values
